# Ansible Take Home Assignment
Interviewee: Aidan Aug

In [1]:
#@title Input

toParse = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
    - Analytics
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes

"""

In [ ]:
#@title Installation
%%capture
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [15]:
import typing
from typing import TypedDict, Literal, Dict, List
from __future__ import annotations

from markdown_it import MarkdownIt
from markdown_it.token import Token
from mdit_py_plugins.tasklists import tasklists_plugin
import re
import json
from datetime import datetime
import os


## Step 1: Authentication

Use Colab's default authentication to handle auth

In [7]:
# Authentication
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

ModuleNotFoundError: No module named 'google.colab'

Alternatively if developing locally, use the following:

In [8]:
#@title Local Authentication

from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

SCOPES = [
    # lets your app create / open / update files it owns or the user selects
    "https://www.googleapis.com/auth/drive.file",

    # full read/write on Google Docs via documents().batchUpdate()
    "https://www.googleapis.com/auth/documents",
]

TOKEN_FILE = "token.json"

creds = None

# Load saved token
if os.path.exists(TOKEN_FILE):
    creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)

# If no token or it's expired, run auth flow
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file("client_secret.json", SCOPES)
        creds = flow.run_local_server(port=0)

    # Save the token
    with open(TOKEN_FILE, "w") as token:
        token.write(creds.to_json())



## Step 2: Parse data

In [9]:
class Location(TypedDict):
    segmentId: str
    index: int

class InsertText(TypedDict):
    text: str
    location: Location

class InsertTextRequest(TypedDict):
    insertText: InsertText

class ParagraphStyle(TypedDict):
    namedStyleType: str

class Range(TypedDict):
    startIndex: int
    endIndex: int

class UpdateParagraphStyle(TypedDict):
    paragraphStyle: ParagraphStyle
    range: Range
    fields: Literal["namedStyleType"]

class StyleParaRequest(TypedDict):
    updateParagraphStyle: UpdateParagraphStyle

class CreateParagraphBullets(TypedDict):
    range: Range
    bulletPreset: str

class BulletsRequest(TypedDict):
    createParagraphBullets: CreateParagraphBullets

# Conversion helpers for requests
def insert_text(text: str, index: int) -> InsertTextRequest:
    return {"insertText": {"text": text, "location": {"segmentId": "", "index": index}}}


def style_para(style: str, start: int, length: int) -> StyleParaRequest:
    return {
        "updateParagraphStyle": {
            "paragraphStyle": {"namedStyleType": style},
            "range": {"startIndex": start, "endIndex": start + length},
            "fields": "namedStyleType",
        }
    }


def bullets(start: int, end: int, preset: str) -> BulletsRequest:
    return {
        "createParagraphBullets": {
            "range": {"startIndex": start, "endIndex": end},
            "bulletPreset": preset,
        }
    }

HEADING_STYLES = {1: "HEADING_1", 2: "HEADING_2", 3: "HEADING_3"}

def bullet_preset(tok: Token) -> str:
    return "BULLET_CHECKBOX" if tok.meta.get("task") else "BULLET_DISC_CIRCLE_SQUARE"


In [16]:
def md_to_requests(md_text: str) -> List[dict]:
    """
    Convert Markdown to Google-Docs batchUpdate requests **while preserving**
    the original counter / tab-indent semantics from the regex implementation.
    """

    md = (
        MarkdownIt("gfm-like")
        .use(tasklists_plugin, enabled=True)    # check-box bullets
    )
    toks: List[Token] = md.parse(md_text)

    reqs: List[dict] = []
    cursor = 1  # Google Docs indices start at 1

    # Stack holds one dict per open list level
    # This enables us to track the number of tabs for each list level
    # {"start": int, "tabs": int, "preset": str|None}
    list_stack: List[dict] = []
    footer_start = 0 # We want different styling for the last paragraph. Once we reach an "hr", we begin tracking the footer_start.

    i = 0
    while i < len(toks):
        tok = toks[i]

        # ─────────────  headings  ─────────────
        if tok.type == "heading_open":
            level = int(tok.tag[1]) # tok.tag is "h1", "h2", "h3", etc.
            inline = toks[i + 1]
            text = inline.content + "\n"

            reqs.append(insert_text(text, cursor))
            reqs.append(style_para(HEADING_STYLES[level], cursor, len(text) - 1))

            cursor += len(text)
            i += 3
            continue

        # ─────────────  bullet lists  ─────────────
        # Bullet lists will have types like:
        # bullet_list_open -> list_item_open -> list_item_close -> bullet_list_close
        # For nested, it will be: bullet_list_open (1) -> list_item_open(1) -> bullet_list_open(2) -> list_item_open(2) -> list_item_close(2) -> bullet_list_close(2) -> list_item_close(1) -> bullet_list_close(1)
        if tok.type == "bullet_list_open":
            # Start a new list
            list_stack.append({"start": cursor, "tabs": 0, "preset": None})
            i += 1
            continue

        if tok.type == "bullet_list_close":
            info = list_stack.pop()
            # Only apply bullets once we close the OUTERMOST list
            if not list_stack:                      # depth == 0
                reqs.append(bullets(info["start"], cursor, info["preset"] or
                                     "BULLET_DISC_CIRCLE_SQUARE"))
                cursor -= info["tabs"]              # remove tab chars stripped by Docs
            else:
                # propagate tab-counts upward so the outermost
                # cursor rollback matches original totTabs logic
                list_stack[-1]["tabs"] += info["tabs"]
            i += 1
            continue

        if tok.type == "list_item_open":
            depth = len(list_stack) - 1             # 0,1,2,...
            indent_tabs = "\t" * depth

            # Detect preset from first item inside *current* list
            cur_list = list_stack[-1]
            if cur_list["preset"] is None:
                cur_list["preset"] = bullet_preset(tok)

            # Plain vs task list → visible text excludes "- " / "- [ ] "
            inline = toks[i + 2]                    # li_open, p_open, inline
            visible = inline.content
            text = f"{indent_tabs}{visible}\n"

            reqs.append(insert_text(text, cursor))
            reqs.append(style_para("NORMAL_TEXT", cursor, len(visible)))

            cur_list["tabs"] += depth               # track tabs for this list level
            cursor += len(text)
            i += 4                                  # li_open, p_open, inline, p_close
            continue

        # ─────────────  ordinary paragraph  ─────────────
        if tok.type == "paragraph_open":
            inline = toks[i + 1]
            text = inline.content + "\n"
            reqs.append(insert_text(text, cursor))
            reqs.append(style_para("NORMAL_TEXT", cursor, len(text) - 1))
            cursor += len(text)
            i += 3
            continue


        if tok.type == "hr":
            # Insert a newline and style the paragraph to look like a divider
            text = "\n"
            reqs.append(insert_text(text, cursor))
            reqs.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": cursor, "endIndex": cursor + 1},
                    "paragraphStyle": {"borderBottom": {
                        "width": {
                            "magnitude": 1,
                            "unit": "PT"
                        },
                        "color": {"color": {"rgbColor": {"red": 0, "green": 0, "blue": 0}}},
                        "dashStyle": "SOLID",
                        "padding": {
                            "magnitude": 3,
                            "unit": "PT"
                        }
                    }},
                    "fields": "borderBottom"
                }
            })
            cursor += len(text)
            i += 1
            footer_start = cursor
            continue

        i += 1

    # ─────────────  end of document  ─────────────
    # Apply the footer style to the last paragraph
    if footer_start:
        reqs.append(style_para("NORMAL_TEXT", footer_start, cursor - footer_start))
        reqs.append({
            "updateTextStyle": {
                "range": {"startIndex": footer_start, "endIndex": cursor},
                "textStyle": {
                    "foregroundColor": {
                        "color": {
                            "rgbColor": {
                                "red": 0.5,
                                "green": 0.5,
                                "blue": 0.5
                            }
                        }
                    },
                    "fontSize": {
                        "magnitude": 10,
                        "unit": "PT"
                    },
                    "italic": True
                },
                "fields": "foregroundColor,fontSize,italic"
            }
        })
    return reqs



In [17]:
def append_mention_styles(requests: List[dict]) -> List[dict]:
    mention_requests = []

    for req in requests:

        if "insertText" in req:
            text = req["insertText"]["text"]
            index = req["insertText"]["location"]["index"]

            # Find all @mentions within the text
            for match in re.finditer(r"@\w[\w.]*", text):  # adjust as needed
                start = index + match.start()
                end = index + match.end()
                mention_requests.append({
                    "updateTextStyle": {
                        "range": {
                            "startIndex": start,
                            "endIndex": end
                        },
                        "textStyle": {
                            "bold": True,
                            "foregroundColor": {
                                "color": {
                                    "rgbColor": {"red": 0.1, "green": 0.4, "blue": 0.8}
                                }
                            }
                        },
                        "fields": "bold,foregroundColor"
                    }
                })

    return requests + mention_requests


## Step 3: Write requests to Google Docs

In [18]:
raw_requests = md_to_requests(toParse)
requests = append_mention_styles(raw_requests)

print(json.dumps(requests, indent=4))

ModuleNotFoundError: Linkify enabled but not installed.

In [ ]:
# Create client to interact with Google Docs
service = build('docs', 'v1')
document = service.documents()

# Create new document
now = datetime.now()
date_str = now.strftime("%Y-%m-%d %H:%M:%S")
title = f"Ansible Health Meeting Notes {now.strftime(date_str)}"

create_req_body = {
  "title": title
}

newDoc = document.create(body=create_req_body, x__xgafv=None).execute()
documentId = newDoc['documentId']

# Update document with styles
doc = document.batchUpdate(documentId=documentId, body={"requests": requests}).execute()
pretty_json = json.dumps(doc['replies'], indent=4)
